# Differences between Rotten Tomatoes' Tomatometer and Audience score - filtered by genre.
Does the difference between critic score and audience score vary by genre. The project's idea is to mine the famous [Rotten Tomatoes](https://www.rottentomatoes.com) for the scores of movies and then look-up the genres in imdb.

Once I got the data I will store it in a MySQL database and query from there.

In [52]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium import webdriver

In [91]:
# Defining the drivers and the URL
driver = webdriver.Firefox()
driver.get('https://www.rottentomatoes.com/browse/dvd-streaming-all/')
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [93]:
# Scraping Functions
def get_movie_count(html):
    init_movie_count = html.find(attrs={"id": "count-link"}).findAll('span')[2].text.split()[1]
    movie_count = html.find(attrs={"id": "count-link"}).findAll('span')[2].text.split()[-1]
    
    return init_movie_count, movie_count

In [92]:
get_movie_count(soup)

('32', '19636')